In [ ]:
#@title blank template
#@markdown This notebook from [github.com/matteoferla/pyrosetta_help](https://github.com/matteoferla/pyrosetta_help).

#@markdown It can be opened in Google Colab via [https://colab.research.google.com/github/matteoferla/pyrosetta_help/blob/main/colabs/colabs-pyrosetta.ipynb](https://colab.research.google.com/github/matteoferla/pyrosetta_help/blob/main/colabs/colabs-pyrosetta.ipynb)

#@markdown It just for loading up PyRosetta

In [ ]:
#@title Installation
# ================================================================

#@markdown Modules installed pyrosetta

# ================================================================
#@markdown Installing PyRosetta with optional backup to your drive (way quicker next time!).
#@markdown Note that PyRosetta occupies some 10 GB, so you'll need to be on the 100 GB plan of Google Drive (it's one pound a month).

#@markdown NB. If `use_drive` is True, you will be prompted to give permission to
#@markdown use Google Drive —_always_ remember to check strangers code against data theft: search and look for all instances of `http`, `requests` and `post` in the code.

#@markdown ### Download PyRosetta
#@markdown The following is not the real password. However, the format is similar.
username = 'boltzmann'  #@param {type:"string"}
password = 'constant'  #@param {type:"string"}
#@markdown Are these the "normal" common credentials?
#@markdown If so their hash will be checked beforehand to check if they are correct
#@markdown (we don't want colab blocked by too many typos).
hash_comparision_required = True  #@param {type:"boolean"}
#@markdown The release to install will be the latest (from [graylab.jhu.edu/download/PyRosetta4](https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python37.ubuntu/) for latest).

#@markdown Use Google Drive for PyRosetta (way faster next time, but takes up space)
#@markdown (NB. You may be prompted to follow a link and possibly authenticate and then copy a code into a box
use_drive = True  #@param {type:"boolean"}
#@markdown Installing `rdkit` and `rdkit_to_params` allows the creation of custom topologies (params) for new ligands
install_rdkit = True  #@param {type:"boolean"}

# ================================================================

import os
from typing import *


# ## Functions

def get_shell_mode() -> str:
    """
    Muppet-proofing: are we in colab?
    """
    # get_ipython is a standard libary in ipython, but not script
    from IPython import get_ipython
    shell_name = get_ipython().__class__.__name__
    if shell_name == 'Shell':
        return 'colab'
    elif shell_name == 'ZMQInteractiveShell':
        return 'jupyter'
    elif shell_name == 'TerminalInteractiveShell':
        raise RuntimeError('This is a colabs notebook. Why are you running it in the terminal?')
    else:
        raise RuntimeError(f'This is a colabs notebook. not a {shell_name}')


def install_and_import(package_name: str,
                       pypi_name: Optional[str] = None,
                       alias_name: Optional[str] = None):
    """If the module has a different name in pypi (`pypi_name`)
    than its import name (`package_name`), specify it.

         pip install pypi_name
         import package_name as alias_name
    """
    import importlib
    # I will go to hell for this, but shmeh:
    from pip._internal.cli.main import main as pip_main
    if pypi_name is None:
        pypi_name = package_name
    if alias_name is None:
        alias_name = package_name
    try:
        importlib.import_module(package_name)
    except ImportError as error:
        if error.name != package_name:
            # these are not the droids we are looking for
            raise ImportError(f'Import of {package_name} requires module {error.name}...',
                              name=error.name)
        pip_main(['install', pypi_name])
    globals()[alias_name] = importlib.import_module(package_name)



def set_workingpath():
    # ## Use drive?
    modality = get_shell_mode()
    if modality != 'colab':
        # jupyter --> stay
        return './'
    elif use_drive:
        from google.colab import drive
        drive.mount('/content/drive')
        return '/content/drive/MyDrive'
        os.chdir(_path)
    else:  # --> stay
        return '/content'

# ## Installer for pyrosetta
def assert_password(username: str, password: str) -> None:
    """
    Verify the username and password are correct without actually knowing them.
    I worry that there may be a large number of idiots that try to guess the passwords
    thus getting Colab fail2ban jailed.
    """
    import hashlib
    hashed_username = hashlib.sha256(username.encode()).hexdigest()
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    expected_hashed_username = 'cf6f296b8145262b22721e52e2edec13ce57af8c6fc990c8ae1a4aa3e50ae40e'
    expected_hashed_password = '45066dd976d8bf0c05dc8dd4d58727945c3437e6eb361ba9870097968db7a0da'
    msg = ('The hash of the {} is not as expected: ' +
           'if your username and password combo are correct and just not the academic ones, ' +
           'set `hash_comparision_required` to False.' +
           'If you dont know the password visit the Rosetta Commons site. ' +
           'The password and username for PyRosetta are DIFFERENT than Rosetta or FoldIt. ' +
           'Please do not google for username:password for this or for anything ' +
           'as stats like number of registered users is mighty important for grants and stuff.')
    assert hashed_username == expected_hashed_username, msg.format('username')
    assert hashed_password == expected_hashed_password, msg.format('password')

def get_latest_release_url(username: str, password: str) -> str:
    # assumes the system is Ubuntu
    import sys, requests, re
    from IPython.display import display, HTML
    py_version = str(sys.version_info.major) + str(sys.version_info.minor)
    base_url = f'https://graylab.jhu.edu/download/PyRosetta4/archive/release/PyRosetta4.Release.python{py_version}.ubuntu'
    url_to_latest = f'{base_url}/latest.html'
    latest_response = requests.get(url_to_latest,
                                   auth=requests.auth.HTTPBasicAuth(username, password)
                                   )
    if latest_response.status_code == 401:
        raise ValueError('Incorrect username or password!')
    elif latest_response.status_code not in (200, 300, 301, 302, 303, 304, 305, 306, 307, 308):
        display(HTML(latest_response.text))
        raise ValueError(f'Something is wrong with the url {url_to_latest}')
    return base_url+'/'+re.search(r'[uU][rR][lL]=(.*?)["\s]', latest_response.text).group(1)

def install_pyrosetta(path:str, username:str='', password:str=''):
    import sys, importlib, site, re
    # is pyrosetta installed?
    if importlib.util.find_spec('pyrosetta'):
        import pyrosetta  # were it not stable, this could raise an import error due to missing requirements
    # is there a Pyrosetta release file in the location?
    elif any(['PyRosetta4.Release' in filename for filename in os.listdir(path) if os.path.isdir(filename)]):
        release_folder = [filename for filename in os.listdir(path)
                          if 'PyRosetta4.Release' in filename and os.path.isdir(filename)][0]
        assert not os.system(f'pip3 install -e {_path}/{release_folder}/setup/')
    # download
    elif not username or not password:
        raise ValueError('Please provide username and password for PyRosetta download')
    else:
        # check if hash is right.
        if hash_comparision_required:
            assert_password(username, password)
        # download tar and uncompress it on the fly
        latest_url = get_latest_release_url(username, password)
        assert not os.system(
            f'curl -u {username}:{password} {latest_url} | tar -xj')
        pyrosetta_folder = re.sub(r'.tar.\w+$', '', os.path.split(latest_url)[1])
        assert not os.system(f'pip3 install -e {_path}/{pyrosetta_folder}/setup/')
    # refresh:
    site.main()

# ========================================================================================
# ## Install stuff
_path = set_workingpath()
install_pyrosetta(_path, username=username, password=password)
install_and_import(package_name='Bio', pypi_name='biopython')
if install_rdkit:
    install_and_import(package_name='rdkit', pypi_name='rdkit-pypi')
    # importing these in main makes them render properly in the notebook:
    from rdkit import Chem
    from rdkit.Chem import PandasTools
install_and_import(package_name='pyrosetta_help', pypi_name='pyrosetta-help', alias_name='ph')
install_and_import(package_name='rdkit_to_params', pypi_name='rdkit-to-params')
install_and_import('py3Dmol')
os.system('pip install --upgrade plotly')
import site
site.main()

In [ ]:
#@title Start PyRosetta
import pyrosetta
import pyrosetta_help as ph

no_optH = False  #@param {type:"boolean"}
ignore_unrecognized_res = False  #@param {type:"boolean"}
load_PDB_components = False  #@param {type:"boolean"}
ignore_waters = False  #@param {type:"boolean"}

extra_options = ph.make_option_string(no_optH=no_optH,
                                      ex1=None,
                                      ex2=None,
                                      mute='all',
                                      ignore_unrecognized_res=ignore_unrecognized_res,
                                      load_PDB_components=load_PDB_components,
                                      ignore_waters=ignore_waters)

# capture to log
logger = ph.configure_logger()
pyrosetta.init(extra_options=extra_options)

In [ ]:
## Usual stuff
pose = ph.pose_from_alphafold2('P02144')

scorefxn = pyrosetta.get_fa_scorefxn()
relax = pyrosetta.rosetta.protocols.relax.FastRelax(scorefxn, 3)
movemap = pyrosetta.MoveMap()
movemap.set_bb(False)
movemap.set_chi(True)
relax.apply(pose)

In [ ]:
# Note that nglview does not work with Colabs but py3Dmol does.
# install py3Dmol
os.system(f'pip3 install py3Dmol')
import site

site.main()
# run
import py3Dmol

view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js', )
view.addModel(ph.get_pdbstr(pose), 'pdb')
view.zoomTo()
view

In [ ]:
# Also note that RDKit Chem.Mol instances are not displays as representations by default.

In [ ]:
#@title Upload to Michelanglo (optional)
#@markdown [Michelanglo](https://michelanglo.sgc.ox.ac.uk/) is a website that
#@markdown allows the creation, annotation and sharing of a webpage with an interactive protein viewport.
#@markdown ([examples](https://michelanglo.sgc.ox.ac.uk/gallery)).
#@markdown The created pages are private —they have a 1 in a quintillion change to be guessed within 5 tries.

#@markdown Registered users (optional) can add interactive annotations to pages.
#@markdown A page created by a guest is editable by registered users with the URL to it
#@markdown (this can be altered in the page settings).
#@markdown Leave blank for guest (it will not add an interactive description):

username = ''  #@param {type:"string"}
password = ''  #@param {type:"string"}

import os

assert not os.system(f'pip3 install michelanglo-api')
import site

site.main()
from michelanglo_api import MikeAPI, Prolink

if not username:
    mike = MikeAPI.guest_login()
else:
    mike = MikeAPI(username, password)

page = mike.convert_pdb(pdbblock=ph.get_pdbstr(pose),
                        data_selection='ligand',
                        data_focus='residue',
                        )
if username:
    page.retrieve()
    page.description = '## Description\n\n'
    page.description += 'autogen bla bla'
    page.commit()
page.show_link()